In [1]:
import os
import math
import shutil
import warnings
import importlib
import urllib.request as request
from contextlib import closing
from numba import cuda

import cupy
import cudf
import cuml

import dask_cudf
import dask_ml
import dask

from cuml.manifold import UMAP as cuUMAP
from cuml.dask.decomposition import PCA as cuDaskPCA
from cuml.dask.cluster import KMeans as cuDaskKMeans
from cuml.dask.manifold import UMAP as cuDaskUMAP

from dask.distributed import Client, LocalCluster
from dask_cuda import initialize, LocalCUDACluster
from dask_cuda.local_cuda_cluster import cuda_visible_devices

from bokeh.io.export import export_png
from bokeh.plotting import figure
from bokeh.models.tickers import FixedTicker
from bokeh.io import output_notebook, push_notebook, show

import nvidia.cheminformatics.chembldata as chembldata

warnings.filterwarnings('ignore', 'Expected ')
warnings.simplefilter('ignore')

output_notebook()

Loading BokehJS ...

### Configurations and settings

In [2]:
# Please add or remove device ids that can be used
# CUDA_VISIBLE_DEVICES=[0]
CUDA_VISIBLE_DEVICES = cuda_visible_devices(0).split(',')

pca_comps = 64
n_clusters = 6
n_neighbors=100
num_mols=5000

enable_tcp_over_ucx = True
enable_nvlink = False
enable_infiniband = False

COLORS = ["#406278", "#e32636", "#9966cc", "#cd9575", "#915c83", "#008000",
          "#ff9966", "#848482", "#8a2be2", "#de5d83", "#800020", "#e97451",
          "#5f9ea0", "#36454f", "#008b8b", "#e9692c", "#f0b98d", "#ef9708",
          "#0fcfc0", "#9cded6", "#d5eae7", "#f3e1eb", "#f6c4e1", "#f79cd4"]
FINGER_PRINT_FILES = 'filter_*.h5'

### Functions

In [3]:
def show_cluster_plot(ldcudf, title='UMAP'):
    """
    Draws a scatter plots from output of UMAP.
    """
    umap_fig = figure(title=title, width=800, output_backend="webgl")
    clusters = ldcudf['cluster'].unique().values_host

    for cluster in clusters:
        query = 'cluster == %s' % (cluster)

        cdf = ldcudf.query(query)

        if cdf.shape[0] == 0:
            continue

        x_array = cupy.fromDlpack(cdf['x'].to_dlpack())
        y_array = cupy.fromDlpack(cdf['y'].to_dlpack())

        umap_fig.circle(x_array.get(),
                        y_array.get(),
                        size=2,
                        color=COLORS[cluster],
                        alpha=0.5, legend = 'Cluster ' + str(cluster))

    umap_fig.legend.location = 'top_right'
    umap_fig.legend.title = 'Clusters'
    
    umap_fig_handle = show(umap_fig, notebook_handle=True)
    push_notebook(handle=umap_fig_handle)
    
def plot_kmeans_umap(df2):
    kmeans_float = cuml.KMeans(n_clusters=n_clusters)
    kmeans_float.fit(df2)

    umap = cuml.UMAP(n_neighbors=100,
                     a=1.0,
                     b=1.0,
                     learning_rate=1.0)
    Xt = umap.fit_transform(df2)

    df2['x'] = Xt[0]
    df2['y'] = Xt[1]
    df2['cluster'] = kmeans_float.labels_

    show_cluster_plot(df2)


# Download ChEMBL database

In [4]:
data_dir = "/data/db"
db_file = os.path.join(data_dir, 'chembl_27.db')

if not os.path.exists(db_file):
    print('Downloading ChEMBL db...')

    os.makedirs(data_dir, exist_ok=True)
    with closing(request.urlopen('ftp://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/chembl_27_sqlite.tar.gz')) as r:
        with open(db_file, 'wb') as f:
            shutil.copyfileobj(r, f)

    print('Download completed')
else:
    print('Reusing available ChEMBL db at', db_file)

Reusing available ChEMBL db at /data/db/chembl_27.db


In [5]:
cluster = LocalCUDACluster(protocol="ucx",
                           dashboard_address=':9001',
                           # TODO: automate visible device list
                           CUDA_VISIBLE_DEVICES=CUDA_VISIBLE_DEVICES,
                           enable_tcp_over_ucx=enable_tcp_over_ucx,
                           enable_nvlink=enable_nvlink,
                           enable_infiniband=enable_infiniband)
client = Client(cluster)
n_workers = len(client.scheduler_info()['workers'].keys())
client

Client Scheduler: ucx://127.0.0.1:36713 Dashboard: http://127.0.0.1:9001/status,Cluster Workers: 1 Cores: 1 Memory: 49.27 GB


# Generate fingerprint from ChEMBL

The 4 in ECFP4 corresponds to the diameter of the atom environments considered, while the Morgan fingerprints take a radius parameter. So a Morgan fingerprint with radius=2 is roughly equivalent to ECFP4 and FCFP4.

In [6]:
%%time
import nvidia.cheminformatics.chembldata
importlib.reload(nvidia.cheminformatics.chembldata)

cache_directory = '/data/fingerprint/'
# cache_directory = None
from nvidia.cheminformatics.fingerprint import MorganFingerprint

if cache_directory is None:
    chem_data = chembldata.ChEmblData(db_file=db_file, fp_type=MorganFingerprint)
    ddf = chem_data.fetch_all_props(num_recs=num_mols)
else:
    hdf_path = os.path.join(cache_directory, FINGER_PRINT_FILES)
    ddf = dask.dataframe.read_hdf(hdf_path, 'fingerprints')

    if num_mols > 0:
        ddf = ddf.head(num_mols, compute=False, npartitions=-1)

dcudf = dask_cudf.from_dask_dataframe(ddf)
dcudf = dcudf.persist()
df = dcudf.compute()

CPU times: user 11.5 s, sys: 966 ms, total: 12.5 s
Wall time: 30 s


In [7]:
# Compute Tanimoto similarities
fp = cupy.fromDlpack(df.to_dlpack())

# Tanimoto Similarity

In [18]:
# Implementation of https://en.wikipedia.org/wiki/Jaccard_index#Other_definitions_of_Tanimoto_distance

@cuda.jit
def compute_norms(data, norms):
    i = cuda.grid(1)
    norms[i] = len(data[i])
    for j in range(len(data[i])):
        if data[i][j] != 0:
            value = j + 1
            data[i][j] = value
            norms[i] = norms[i] + (value**2)
    
    if norms[i] != 0:
        norms[i] = math.sqrt(norms[i])

        
@cuda.jit
def compute_tanimoto_sim_matix(data, norms, sim_array):
    x = cuda.grid(1)
    rows = len(data)
    
    i = x // rows
    j = x % rows
    
    if i == j:
        sim_array[i][j] = 1
        return
    
    a = data[i]
    b = data[j]
    
    prod = 0
    for k in range(len(data[i])):
        prod = prod + (a[k] * b[k])
        
    a_norm = norms[i]
    b_norm = norms[j]
    
    sim_array[i][j] = (prod / ((a_norm**2 + b_norm**2) - prod))
    
    
@cuda.jit
def compute_tanimoto_dist_matix(data, norms, sim_array):
    x = cuda.grid(1)
    rows = len(data)
    
    i = x // rows
    j = x % rows
    
    if i == j:
        sim_array[i][j] = 0
        return
    
    a = data[i]
    b = data[j]
    
    prod = 0
    for k in range(len(data[i])):
        prod = prod + (a[k] * b[k])
        
    a_norm = norms[i]
    b_norm = norms[j]
    
    sim_array[i][j] = 1.0 - (prod / ((a_norm**2 + b_norm**2) - prod))
    
    
def tanimotoSimilarity(fp):
    norms = cupy.zeros(fp.shape[0])
    compute_norms.forall(norms.shape[0], 1)(fp, norms)

    sim_array = cupy.zeros((fp.shape[0], fp.shape[0]), cupy.float32)
    compute_tanimoto_sim_matix.forall(fp.shape[0] * fp.shape[0], 1)(fp, norms, sim_array)
    return sim_array


def tanimotoDistance(fp):
    norms = cupy.zeros(fp.shape[0])
    compute_norms.forall(norms.shape[0], 1)(fp, norms)

    sim_array = cupy.zeros((fp.shape[0], fp.shape[0]), cupy.float32)
    compute_tanimoto_dist_matix.forall(fp.shape[0] * fp.shape[0], 1)(fp, norms, sim_array)
    return sim_array

# Jaccard Index Similarity

In [19]:
# Implementation of https://en.wikipedia.org/wiki/Jaccard_index
@cuda.jit
def compute_jaccard_index_matix(data, index_array):
    x = cuda.grid(1)
    rows = len(data)
    
    i = x // rows
    j = x % rows
    
    a = data[i]
    b = data[j]
    
    intersects = 0
    unions = 0
    for k in range(len(data[i])):
        if a[k] == 1 and b[k] == 1:
            intersects += 1
            unions += 1
        elif a[k] == 1 or b[k] == 1:
            unions += 1

    index_array[i][j] = (intersects / unions)

def jaccardIndex(fp):
    index_array = cupy.zeros((fp.shape[0], fp.shape[0]), cupy.float32)
    compute_jaccard_index_matix.forall(fp.shape[0] * fp.shape[0], 1)(fp, index_array)
    return index_array

# Clustering

In [20]:
%%time
from cuml.cluster import DBSCAN

dist_array = tanimotoDistance(fp)
sim_array = tanimotoSimilarity(fp)

# sim_array = jaccardIndex(fp)
# plot_kmeans_umap(df2)

CPU times: user 3.16 s, sys: 1.32 s, total: 4.48 s
Wall time: 4.4 s


In [198]:
df2 = cudf.DataFrame(sim_array)
def plot_kmeans_umap(df2):
    kmeans_float = cuml.KMeans(n_clusters=n_clusters, 
                               random_state=0,
                              init='k-means||')
    kmeans_float.fit(df2)
#     kmeans_float = DBSCAN(eps = 1.0, min_samples = 1)
#     kmeans_float.fit(df2)

    umap = cuml.UMAP(n_neighbors=100,
                     min_dist=0.2,
                     spread=3.0,
                     a=1,
                     b=1,
                     learning_rate=1.0)
    Xt = umap.fit_transform(df2)

    df2['x'] = Xt[0]
    df2['y'] = Xt[1]
    df2['cluster'] = kmeans_float.labels_
    show_cluster_plot(df2)
    
    return  kmeans_float.cluster_centers_

cluster_centers = plot_kmeans_umap(df2)

## Silhouette Score Implementation

In [224]:
from sklearn.metrics.cluster import silhouette_score, silhouette_samples

In [225]:
# Get cluster labels and the centers for calculation -- this repeats calculation above in function
kmeans_float = cuml.KMeans(n_clusters=n_clusters, 
                           random_state=0,
                           init='k-means||')

clusters = kmeans_float.fit(cudf.DataFrame(dist_array)).labels_
cluster_centers = kmeans_float.cluster_centers_

cluster_centers_arr = cupy.fromDlpack(cluster_centers.to_dlpack())
clusters_arr = cupy.fromDlpack(clusters.to_dlpack())

In [226]:
%%time
silhouette_score(cupy.asnumpy(dist_array), cupy.asnumpy(clusters_arr))

CPU times: user 8.86 s, sys: 3.67 s, total: 12.5 s
Wall time: 1.27 s


0.080111

In [207]:
@cuda.jit()
def distance_to_cluster_neighbors(data, data_clusters, lookup_clusters, out):
    x = cuda.grid(1)
    n_data = data.shape[0]
    n_features = data.shape[1]
    
    lookup_cluster = lookup_clusters[x]
    total_distance = 0.0
    n_distances = 0
    for idx in range(n_data):
        if (idx != x) & (data_clusters[idx] == lookup_cluster):
            dist = 0.0
            for feat in range(n_features):
                dist += math.pow(data[x, feat] - data[idx, feat], 2)
            total_distance += math.sqrt(dist)
            n_distances += 1
            
    out[x] = total_distance / n_distances

@cuda.jit()
def pairwise_euclidean_distance(data, out):
    row = cuda.grid(1)
    n_data = data.shape[0]
    n_features = data.shape[1]
    
    for col in range(row + 1, n_data):
        dist = 0.0
        for feat in range(n_features):
            dist += math.pow(data[row, feat] - data[col, feat], 2)
        dist = math.sqrt(dist)
        out[row, col] = dist
        out[col, row] = dist

        
def kneighbors(data, n_neighbors=None):
    n_data = data.shape[0]
    
    if n_neighbors is None:
        n_neighbors = n_data - 1
    else:
        assert n_neighbors < n_data
    
    # Calculate pairwise distances
    pairwise_dist = cupy.zeros((n_data, n_data), cupy.float32)
    pairwise_euclidean_distance.forall(data.shape[0], 1)(data, pairwise_dist)
    
    # Get the ID of the nearest neighbors
    cupy.fill_diagonal(pairwise_dist, cupy.inf)
    nearest_neighbors = pairwise_dist.argsort(axis=1)

    return nearest_neighbors[:, :n_neighbors]

In [213]:
%%time

def get_silhouette_score(dist_array, cluster_centers, clusters):
    n_data = dist_array.shape[0]

    # Calculate the intra-cluster distance (variable "a")
    intra_cluster_dist = cupy.zeros(n_data, cupy.float32)
    distance_to_cluster_neighbors.forall(n_data, 1)(dist_array, clusters, clusters, intra_cluster_dist)

    # Calculate the nearest-cluster distance to next nearest cluster (variable "b")
    # First requires getting the nearest cluster for each data point
    # Then nearest-cluster distance is almost identical to previous but with different cluster IDs

    nearest_neighbor = kneighbors_from_distance(cluster_centers, 1).squeeze()
    nearest_neighbor_cluster = nearest_neighbor[clusters]
    nearest_cluster_dist = cupy.zeros(n_data, cupy.float32)
    distance_to_cluster_neighbors.forall(n_data, 1)(dist_array, clusters, nearest_neighbor_cluster, nearest_cluster_dist)

    numerator   = nearest_cluster_dist - intra_cluster_dist
    denominator = cupy.vstack([nearest_cluster_dist, intra_cluster_dist]).max(axis=0)
    silhouette_samples = numerator / denominator
    
    return silhouette_samples.mean()

silhouette_score = get_silhouette_score(dist_array, cluster_centers_arr, clusters_arr)
silhouette_score

CPU times: user 27.3 s, sys: 12.8 s, total: 40.1 s
Wall time: 38.7 s


array(0.10220183, dtype=float32)

This is a faster version that calculates both the inter cluster and nearest cluster distance at once. However there appears to be some issues with the code.

In [214]:
%%time

@cuda.jit()
def distance_to_cluster_neighbors_2(data, data_clusters, neighbor_clusters, out):
    x = cuda.grid(1)
    n_data = data.shape[0]
    n_features = data.shape[1]
    
    id_cluster = data_clusters[x]
    id_total_distance = 0.0
    id_n_distances = 0
    
    neighbor_cluster = neighbor_clusters[x]
    neighbor_total_distance = 0.0
    neighbor_n_distances = 0
    
    for idx in range(n_data):
        id_dist, neighbor_dist = 0.0, 0.0
        if (idx != x):
            if (data_clusters[idx] == id_cluster):  
                for feat in range(n_features):
                    id_dist += math.pow(data[x, feat] - data[idx, feat], 2)
                id_total_distance += math.sqrt(id_dist)
                id_n_distances += 1
            elif (data_clusters[idx] == neighbor_cluster):
                for feat in range(n_features):
                    neighbor_dist += math.pow(data[x, feat] - data[idx, feat], 2)
                neighbor_total_distance += math.sqrt(neighbor_dist)
                        
    out[x, 0] = id_total_distance / id_n_distances
    out[x, 1] = neighbor_total_distance / neighbor_n_distances
    
    
def get_silhouette_score_2(dist_array, cluster_centers, clusters):
    n_data = dist_array.shape[0]
    nearest_neighbor = kneighbors_from_distance(cluster_centers, 1).squeeze()
    nearest_neighbor_cluster = nearest_neighbor[clusters]
    
    # Calculate the intra-cluster distance (variable "a") and nearest-cluster distance to next nearest cluster (variable "b")
    # This is done in one pass
    intra_nearest_cluster_dist = cupy.zeros((n_data, 2), cupy.float32)
    distance_to_cluster_neighbors_2.forall(n_data, 1)(dist_array, clusters, nearest_neighbor_cluster, intra_nearest_cluster_dist)
    intra_cluster_dist = intra_nearest_cluster_dist[:, 0].squeeze()
    nearest_cluster_dist = intra_nearest_cluster_dist[:, 1].squeeze()

    numerator   = nearest_cluster_dist - intra_cluster_dist
    denominator = cupy.vstack([nearest_cluster_dist, intra_cluster_dist]).max(axis=0)
    silhouette_samples = numerator / denominator    
    return silhouette_samples.mean()

silhouette_score_2 = get_silhouette_score_2(dist_array, cluster_centers_arr, clusters_arr)
silhouette_score_2

CPU times: user 14.5 s, sys: 6.43 s, total: 20.9 s
Wall time: 20.2 s


array(-1., dtype=float32)